# Testing Bennett's Data

`ITP_CourseArtifacts_June 2021_END_of_Course_DeIDENTIFIED.xlsx` using the `Course Meta SelfEff` sheet

In [ ]:
import os
import pandas as pd
import nltk
import seaborn as sns

In [ ]:
%load_ext autoreload
%autoreload 2

from latentscope_helper import latentscope_helper

## Read in and clean the data

These next two cells only need to be run once.  (If rerunning this notebook, you can start after the next markdown cell.)

In [ ]:
# set this to True if you want to change the data the is used 
read_in_original_data_file = False

In [ ]:
# Function to split text into sentences
def split_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

In [ ]:
if (read_in_original_data_file):
    # read in the data
    df = pd.read_excel("../../data/ITP_CourseArtifacts_June 2021_END_of_Course_DeIDENTIFIED.xlsx", sheet_name = "Course Meta SelfEff")
    print(f"length of original DataFrame = {len(df)}")

    # save the question
    question = df.columns[1]
    # only take the columns we need and rename them to remove spaces and special characters
    data_tmp = df.rename(columns = {question:'student_responses'})

    # remove extra newlines
    data_tmp['student_responses'] = data_tmp['student_responses'].str.replace('\n', ' ')
    
    # remove rows with short answers (otherwise the sentence finder might choke -- not sure why)
    n_min = 5
    data_tmp = data_tmp[data_tmp['student_responses'].str.split().str.len().gt(n_min)]  

    # save to .csv file
    data_tmp.to_csv("../../data/ITP_CourseArtifacts_June_2021_END_of_Course_DeIDENTIFIED_Course_Meta_SelfEff_cleaned.csv", index=False)

    # split into sentences
    data = pd.DataFrame(columns=data_tmp.columns)
    for index, row in data_tmp.iterrows():
        # Split the response into sentences
        sentences = split_into_sentences(row['student_responses'])
        
        # Create a new row for each sentence and append it to the new DataFrame
        for sentence in sentences:
            new_row = row.copy()
            new_row['student_responses'] = sentence
            data = data._append(new_row, ignore_index=True)

    # remove rows with short answers (again)
    n_min = 5
    data = data[data['student_responses'].str.split().str.len().gt(n_min)]  

    print(f"length of new DataFrame (after cleaning and sentence splitting) = {len(data)}")

    # Save the new DataFrame to a new file (since this takes a while to run)
    data.to_csv("../../data/ITP_CourseArtifacts_June_2021_END_of_Course_DeIDENTIFIED_Course_Meta_SelfEff_cleaned_split_into_sentences.csv", index=False)


## Initialize and run `latent-scope` using my Class

In [ ]:
# This question has multiple components; so definitely best to take the version with split sentences
data = pd.read_csv("../../data/ITP_CourseArtifacts_June_2021_END_of_Course_DeIDENTIFIED_Course_Meta_SelfEff_cleaned_split_into_sentences.csv")

# initialize my helper object
worker = latentscope_helper(
    latent_scope_dir = "../../latent-scope_data", # directory where the latentscope files are stored
    dataset_id = "ITP_CourseArtifacts_June_2021_END_of_Course_DeIDENTIFIED_Course_Meta_SelfEff", # data set name, for the sub-directory name within latent_scope_dir for this project
    data = data, # pandas DataFrame that contains the data to analyze
    text_column = "student_responses", # response column name from data_file
    remove_old_files = True, # set this to True if you want to clean the latent-scope directories and start fresh
    quick_clean = True, # set this to True if you want to remove every file in the directories, regardless of imin and imax
    imin = 0, # minimum number of for files to search through (to remove)
    imax = 50, # maximum number of for files to search through (to remove)
    run_embedding = True, # whether to run the embedding step (and potentially remove previous files)
    run_umap = True, # whether to run the umap step (and potentially remove previous files)
    run_cluster = True, # whether to run the clustering step (and potentially remove previous files)
    run_label = True, # whether to run the labeling step (and potentially remove previous files)
    embedding_model_id = "transformers-jinaai___jina-embeddings-v2-small-en", # embeddings model name
    embedding_n_dimensions = 512, # number of dimensions for embedding.  reading the jina docs, they often use this number as an example for the number of dimensions (not sure this is a recommendation though)
    umap_n_components = 2, # number of UMAP dimensions
    umap_n_neighbors = 10, # "controls how UMAP balances local versus global structure in the data." Larger values mean UMAP will look at larger distances for neighbors (15 is default)
    umap_min_dist = 0, # "controls how tightly UMAP is allowed to pack points together" (default is 0.1)
    cluster_samples = 5, # min_cluster_size in HDBSCAN : "the smallest size grouping that you wish to consider a cluster"
    cluster_min_samples = 12, # min_samples in HDBSCAN : "provide a measure of how conservative you want your clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas."
    cluster_selection_epsilon =  0.05, # cluster_selection_epsilon in HDBSCAN : distance measure between clusters that "ensures that clusters below the given threshold are not split up any further"
    chat_model_id = "transformers-TinyLlama___TinyLlama-1.1B-Chat-v1.0", # LLM to use for labeling the clusters
    label_length = 10, # max length to tell the LLM to use when generating a given label (not always respected by the LLM!)
    chat_model_instructions_before = "Below is a list of items each starting with [item].  Each item is a response from a different person to a survey. These items all have a similar theme.  The list begins below.", # string of text to provide the LLM as instructions before the list of cluster items is given
    chat_model_instructions_after = "That was the last item in the list.  Now return a concise label for the items in this list that describes the theme.  This label should not be fully verbatim text from any individual item.  Your label should contain no more than 10 words.", # string of text to provide the LLM as instructions after the list of cluster items is given
    scope_description = "First full test with responses separated into sentences", #label to give the scope (when using the latentscope server to view the data)
)

In [ ]:
# run latent-scope (using the inputs from above)

# if embeddings already exist, no need to run them again (unless indending to change embedding parameters)
# worker.run_embedding = False

worker.save_scope = False # I will not save this one, but I will save the scope below after the metrics

worker.initialize_files_and_numbering()
worker.initialize_latentscope()
worker.run_latentscope()

## Calculate metrics to assess the quality of this analysis

Ideally, I would want to do this for a number of runs each changing some parameter and returning a different number of clusters.

In [ ]:

# returns inertia, Silhouette Coefficient, Calinski-Harabasz Index, Davies-Bouldin Index
# - a lower inertia value is generally better
# - a higher Silhouette Coefficient score relates to a model with better defined clusters. 
# - a higher Calinski-Harabasz score relates to a model with better defined clusters.
# - a lower Davies-Bouldin index relates to a model with better separation between the clusters.
worker.calculate_metrics(embedding_number = '00001', umap_number = '00001', cluster_number = '00001')

In [ ]:
# remove some files?
# worker.remove_old_files = True
# worker.suppress_helper_output = False
# worker.imax = 999
# worker.imin = 2
# worker.initialize_files_and_numbering()


In [ ]:
#%%capture
# %%capture magic to suppress output
# TO DO: Figure out how to supress the figures!


# a test looping over one HDBSCAN parameter to check the resulting metrics
worker.suppress_latentscope_output = True
worker.suppress_helper_output = True
worker.remove_old_files = False
worker.initialize_files_and_numbering()

worker.embedding_number = '00001'
worker.run_embedding = worker.run_umap = worker.run_label = worker.run_cluster = worker.save_scope = False

umap_n_components = [2, 3]
umap_n_neighbors = [5, 10, 15, 20, 25, 30]
umap_min_dist = [0, 0.05, 0.1, 0.15]

cluster_min_samples = [5, 10, 15, 20, 25, 30]
cluster_selection_epsilon = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

un = 2 # starting number for umap files
cn = 2 # starting number for the cluster files

cluster_file_numbers = []
umap_file_numbers = []

for u_nc in umap_n_components:
    for u_nn in umap_n_neighbors:
        for u_md in umap_min_dist:
            print(f'umap_file_number = {un}, n_components = {u_nc}, n_neighbors = {u_nn}, min_dist = {u_md}')
            worker.umap_n_components = u_nc
            worker.umap_n_neighbors = u_nn
            worker.umap_min_dist = u_md
            worker.umap_number = str(un).zfill(5)
            worker.run_umap = True
            worker.run_cluster = False
            worker.initialize_latentscope_filenames()
            worker.run_latentscope()

            
            # cluster parameters
            for c_ms in cluster_min_samples:
                for c_eps in cluster_selection_epsilon:
                    print(f'cluster_file_number = {cn}, min_samples = {c_ms}, selection_epsilon = {c_eps}')
                    worker.cluster_min_samples = c_ms
                    worker.cluster_selection_epsilon = c_eps
                    worker.cluster_number = str(cn).zfill(5)
                    worker.run_umap = False
                    worker.run_cluster = True
                    worker.initialize_latentscope_filenames()
                    worker.run_latentscope()
        
                    cluster_file_numbers.append(cn)
                    umap_file_numbers.append(un)
                    cn += 1
                    
            un += 1
            print('')

In [ ]:
# # in case I need to recreate the file numbering structure
# for u_nc in umap_n_components:
#     for u_nn in umap_n_neighbors:
#         for u_md in umap_min_dist:
#             print(f'umap_file_number = {un}, n_components = {u_nc}, n_neighbors = {u_nn}, min_dist = {u_md}')
            
#             # cluster parameters
#             for c_ms in cluster_min_samples:
#                 for c_eps in cluster_selection_epsilon:
#                     print(f'cluster_file_number = {cn}, min_samples = {c_ms}, selection_epsilon = {c_eps}')
        
#                     cluster_file_numbers.append(cn)
#                     umap_file_numbers.append(un)
#                     cn += 1
                    
#             un += 1
#             print('')

In [ ]:
metrics = []
inertia = []
sc = []
ch = []
db = []
n_neighbors = []
min_dist = []
n_components = []
min_samples = []
selection_epsilon = []
n_clusters = []
for (un,cn) in zip(umap_file_numbers, cluster_file_numbers):
    print(un, cn)
    m = worker.calculate_metrics(embedding_number = '00001', umap_number = str(un).zfill(5), cluster_number = str(cn).zfill(5))
    metrics.append(m)
    inertia.append(m['inertia'])
    sc.append(m['silhouette_coefficient'])
    ch.append(m['calinski_harabasz_index'])
    db.append(m['davies_bouldin_index'])
    n_components.append(m['umap_info']['n_components'])
    n_neighbors.append(m['umap_info']['neighbors'])
    min_dist.append(m['umap_info']['min_dist'])
    min_samples.append(m['cluster_info']['min_samples'])
    selection_epsilon.append(m['cluster_info']['cluster_selection_epsilon'])
    n_clusters.append(m['cluster_info']['n_clusters'])
df = pd.DataFrame()
df['umap_file_number'] = umap_file_numbers
df['cluster_file_number'] = cluster_file_numbers
df['umap_n_components'] = n_components
df['umap_n_neighbors'] = n_neighbors
df['umap_min_dist'] = min_dist
df['cluster_min_samples'] = min_samples
df['cluster_selection_epsilon'] = selection_epsilon
df['n_clusters'] = n_clusters
df['inertia'] = inertia
df['silhouette_coefficient'] = sc
df['calinski_harabasz_index'] = ch
df['davies_bouldin_index'] = db

df.to_csv(os.path.join('tables', worker.dataset_id + '_metrics_grid.csv'), index = False)

df



In [ ]:
# I'm still having trouble resetting the plotting backend given the suppression above
import matplotlib
%matplotlib inline

In [ ]:
dfm = pd.read_csv(os.path.join('tables', worker.dataset_id + '_metrics_grid.csv'))
dfm

In [ ]:
cols_to_plot = dfm.columns[2:].tolist()
g = sns.pairplot(dfm[cols_to_plot], corner = True, diag_kind='kde')

In [ ]:
x_vars = ['umap_n_components','umap_n_neighbors','umap_min_dist','cluster_min_samples','cluster_selection_epsilon','n_clusters']
y_vars = ['silhouette_coefficient','calinski_harabasz_index','davies_bouldin_index', 'inertia']
g = sns.PairGrid(dfm, x_vars = x_vars, y_vars = y_vars, hue = 'n_clusters', palette = 'viridis')
g.map(sns.scatterplot)
g.map_diag(sns.histplot)

# I played around with the loc statement to isolate what appears to be the best clusters
# (and asked ChatGPT how to create this overlay)
dfm_best = df.loc[(dfm['n_clusters'] > 5) & (df['n_clusters'] < 30) & (df['silhouette_coefficient'] > 0.4) & (df['calinski_harabasz_index'] > 2000) & (df['davies_bouldin_index'] <2)]
def custom_scatter(x, y, **kwargs):
    sns.scatterplot(x=x, y=y, **kwargs)
    sns.scatterplot(data=dfm_best, x=x.name, y=y.name, marker='o', color='black', edgecolor='black', facecolors='none', s=100)
g.map(custom_scatter)

g.savefig(os.path.join('plots', worker.dataset_id + '_metric_grid.png'), bbox_inches = 'tight') 



In [ ]:
dfm_best

## Pick one of these to create labels and save the scope

In [ ]:

worker.suppress_latentscope_output = False
worker.suppress_helper_output = False
worker.remove_old_files = False
worker.embedding_number = '00001'
worker.umap_number = '00006'
worker.cluster_number = '00276'
worker.scope_number = '00001'

worker.run_embedding = worker.run_umap = worker.run_label = worker.run_cluster = False
worker.run_label = worker.save_scope = True

worker.initialize_files_and_numbering()
worker.initialize_latentscope()
worker.run_latentscope()

## Print and save the output

In [ ]:
# if you want to plot a specific scope number, you can define it here (and you don't need to actually run latentscope in the previous cell)
# worker.remove_old_files = False
# worker.scope_number = '00001'
# worker.initialize_files_and_numbering()
# worker.initialize_latentscope()

In [ ]:
# print the labels
# self.scope_labels_id = '00001'
worker.print_labels()

In [ ]:
# create a bar chart of the labels
f, ax = worker.create_bar_chart(filename = os.path.join('plots', worker.dataset_id + '_scope' + worker.scope_number + '.png'))

In [ ]:
# create an Excel workbook to review the results
# The first sheet will have the raw data.  
# The second sheet will have a map between cluster label and sheet name.  
# Subsequent sheets will be one per cluster containing the cluster data.
data_raw = pd.read_excel("../../data/ITP_CourseArtifacts_June 2021_END_of_Course_DeIDENTIFIED.xlsx", sheet_name = "Course Meta SelfEff")
worker.create_excel_workbook(data_raw, os.path.join('tables', worker.dataset_id + '_clusters_scope' + worker.scope_number + '.xlsx'))

## Run the server to investigate and visualize these results

In [ ]:
#ls.serve()